In [1]:
import pandas as pd 
import os 

In [2]:
folder_path = r'D:\Studia\4 rok\Przestrzenna\Projekt\Dane\var_data'


for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Wczytaj plik CSV do ramki danych pandas, wybierając tylko kolumny "geo", "TIME_PERIOD" i "OBS_VALUE"
        df = pd.read_csv(file_path, usecols=["geo", "TIME_PERIOD", "OBS_VALUE"])
        
        # Pobierz nazwę pliku bez rozszerzenia
        df_name = os.path.splitext(filename)[0]
        
        # Zmiana nazwy kolumny "OBS_VALUE" na nazwę pliku bez rozszerzenia
        df.rename(columns={"OBS_VALUE": df_name, 'TIME_PERIOD': 'time'}, inplace=True)   
               
        globals()[df_name] = df     
        

In [3]:
dfs = [ education, emp_high_tech, gdp, inv,  migration, pop, pop_dens, tourists,unemp, wages ]

# Połącz ramki danych na zasadzie left join
merged_df = pd.DataFrame(dfs[0])  # Rozpocznij od pierwszej ramki danych
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['geo', 'time'], how='left')

# Wyświetl wynik lub zapisz do nowego pliku CSV
print(merged_df)

       geo  time  education  emp_high_tech      gdp     inv  migration  \
0     AT11  2011       14.8            3.4  22100.0  1716.0        7.3   
1     AT11  2012       13.9            3.1  23400.0  1943.0        7.2   
2     AT11  2013       15.8            NaN  23800.0  1838.0        6.4   
3     AT11  2014       25.7            2.3  24100.0  1894.0        6.8   
4     AT11  2015       28.5            2.6  25000.0  1897.0       12.8   
...    ...   ...        ...            ...      ...     ...        ...   
3728  UKN0  2015       33.6            2.7      NaN     NaN        1.1   
3729  UKN0  2016       34.4            2.8      NaN     NaN        0.5   
3730  UKN0  2017       35.6            2.5      NaN     NaN        1.2   
3731  UKN0  2018       37.2            3.3      NaN     NaN        1.2   
3732  UKN0  2019       40.6            3.9      NaN     NaN        NaN   

         pop  pop_dens   tourists  unemp   wages  
0        NaN      77.7        NaN    3.8  3289.0  
1        

In [14]:
merged_df = merged_df[(merged_df['time'] > 2013) & (merged_df['time'] <= 2019)]
merged_df

,geo,time,education,emp_high_tech,gdp,inv,migration,pop,pop_dens,tourists,unemp,wages
3,AT11,2014,25.7,2.3,24100.0,1894.0,6.8,189.3,78.5,158096.0,4.8,3707.0
4,AT11,2015,28.5,2.6,25000.0,1897.0,12.8,189.2,79.0,159078.0,5.2,3793.0
5,AT11,2016,27.9,3.0,25600.0,1894.0,6.3,189.5,79.4,167212.0,5.7,3917.0
6,AT11,2017,28.0,3.2,26400.0,2192.0,6.5,188.7,79.7,172673.0,5.2,4084.0
7,AT11,2018,28.4,3.8,27100.0,2225.0,6.3,188.1,77.5,178502.0,4.2,4299.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3728,UKN0,2015,33.6,2.7,NaN,NaN,1.1,1181.6,136.5,1000850.0,6.1,NaN
3729,UKN0,2016,34.4,2.8,NaN,NaN,0.5,1182.2,137.3,407230.0,5.7,NaN
3730,UKN0,2017,35.6,2.5,NaN,NaN,1.2,1180.1,138.0,704040.0,4.5,NaN
3731,UKN0,2018,37.2,3.3,NaN,NaN,1.2,1177.3,138.7,704040.0,3.6,NaN


In [19]:
df=merged_df
for column in df.columns.drop(['geo', 'time']):
    # Group by 'geo' and calculate median for each group
    median_values = df.groupby('geo')[column].median()

    # Function to apply for each row to fill missing values
    def fill_with_median(row, col_name):
        if pd.isnull(row[col_name]):
            return median_values[row['geo']]
        else:
            return row[col_name]

    # Applying the function to the DataFrame
    df[column] = df.apply(lambda row: fill_with_median(row, column), axis=1)

# Verifying if all missing values are filled
df.isnull().sum()

geo                0
time               0
education          0
emp_high_tech     78
gdp              268
inv              472
migration         30
pop                0
pop_dens          30
tourists          90
unemp              6
wages            496
dtype: int64

In [21]:
# Filling missing values with the global mean of each column, excluding missing values from the calculation

for column in df.columns.drop(['geo', 'time', ]):
    # Calculate the mean for each column excluding NaN values
    mean_value = df[column].mean(skipna=True)
    # Fill missing values with the calculated mean
    df[column].fillna(mean_value, inplace=True)

# Verifying if all missing values are filled
df.isnull().sum()

geo              0
time             0
education        0
emp_high_tech    0
gdp              0
inv              0
migration        0
pop              0
pop_dens         0
tourists         0
unemp            0
wages            0
dtype: int64

In [27]:
# Znalezienie unikalnych wartości 'geo' i 'time'
unique_geo = df['geo'].unique()
unique_years = df['time'].unique()

# Znalezienie brakujących kombinacji
missing_combinations = []
for geo in unique_geo:
    for year in unique_years:
        if not ((df['geo'] == geo) & (df['time'] == year)).any():
            missing_combinations.append((geo, year))

# Tworzenie DataFrame dla brakujących kombinacji
missing_combinations_df = pd.DataFrame(missing_combinations, columns=['geo', 'time'])

# Uzupełnianie brakujących danych globalną średnią
for column in df.columns.drop(['geo', 'time']):
    mean_value = df[column].mean(skipna=True)
    missing_combinations_df[column] = mean_value

# Dołączanie brakujących kombinacji do oryginalnego DataFrame
df_completed = df.append(missing_combinations_df, ignore_index=True)
df_completed.isnull().sum()

geo              0
time             0
education        0
emp_high_tech    0
gdp              0
inv              0
migration        0
pop              0
pop_dens         0
tourists         0
unemp            0
wages            0
dtype: int64

In [28]:
df_completed.to_csv(r'D:\Studia\4 rok\Przestrzenna\Projekt\Dane\dataset2.csv', index=False)

In [ ]:
df = merged_df
kompletny_zakres_lat = list(range(2014, 2019))

# Grupowanie danych według kolumny "geo" i sprawdzenie, czy dla każdego "geo" są wszystkie lata od 2011 do 2019
brakujace_lata = df.groupby('geo')['time'].apply(lambda x: list(set(kompletny_zakres_lat) - set(x)))

# Wyświetlenie unikalnych "geo" dla których brakują lata
brakujace_lata_geo = brakujace_lata[brakujace_lata.apply(lambda x: len(x) > 0)].index.tolist()

# Utworzenie nowego dataframe, w którym dla brakujących lat dodane zostaną brakujące wiersze
df_uaktualniony = pd.DataFrame()
for geo in brakujace_lata_geo:
    brakujace_wiersze = pd.DataFrame({'geo': [geo] * len(brakujace_lata[geo]), 'time': brakujace_lata[geo]})
    df_uaktualniony = pd.concat([df_uaktualniony, brakujace_wiersze])

# Utworzenie pełnego dataframe z aktualnymi danymi
df_uaktualniony = pd.concat([df, df_uaktualniony], ignore_index=True)

# Sortowanie dataframe według kolumn "geo" i "time"
df_uaktualniony = df_uaktualniony.sort_values(by=['geo', 'time']).reset_index(drop=True)

# Wyświetlenie zaktualizowanego dataframe
print(df_uaktualniony)

In [23]:
merged_df.sort_values(by='emp_high_tech', ascending=False)

,geo,time,education,emp_high_tech,gdp,inv,migration,pop,pop_dens,tourists,unemp,wages
3599,UKJ1,2019,53.0,11.9,26624.823944,10958.438373,2.800000,1544.8,414.100000,1891946.5,2.8,25124.921721
1815,HR05,2019,42.8,11.2,38800.000000,3907.840000,2.870906,521.2,444.525541,1210921.0,5.4,8768.800000
3594,UKJ1,2014,51.0,11.0,26624.823944,10958.438373,4.100000,1509.2,407.000000,1891946.5,4.2,25124.921721
3597,UKJ1,2017,53.9,10.9,26624.823944,10958.438373,2.800000,1504.9,417.000000,1891946.5,2.9,25124.921721
3596,UKJ1,2016,51.4,10.9,26624.823944,10958.438373,2.200000,1498.4,414.100000,2069561.0,2.7,25124.921721
...,...,...,...,...,...,...,...,...,...,...,...,...
3232,TR63,2017,12.1,0.2,12300.000000,10958.438373,-1.300000,2015.3,139.000000,68330.0,11.5,25124.921721
3231,TR63,2016,12.8,0.2,11600.000000,10958.438373,0.100000,1944.2,137.100000,68330.0,14.4,25124.921721
3343,TRC1,2018,13.4,0.2,12500.000000,10958.438373,-3.800000,1690.9,183.000000,129201.0,12.3,25124.921721
3311,TRA2,2016,8.9,0.1,8300.000000,10958.438373,-27.100000,692.7,38.000000,46533.0,4.8,25124.921721


In [ ]:
def impute_row(row):
    means = row.iloc[2:].mean()  # Uzyskaj średnie dla każdej kolumny, zaczynając od trzeciej kolumny
    return row.fillna(means)  # Uzupełnij braki danych średnimi

# Uzupełnij braki w danych dla każdego wiersza
merged_df = merged_df.apply(impute_row, axis=1)

def round_column(column):
    return column.apply(lambda x: round(x, 2) if pd.notna(x) else x)

# Zaokrąglij wartości od drugiej kolumny włącznie
merged_df = merged_df.apply(lambda col: round_column(col) if col.name in df.columns[2:] else col, axis=0)


# Wyświetl wynik lub zapisz do nowego pliku CSV
print(merged_df)

In [ ]:
merged_df.to_csv(r'D:\Studia\4 rok\Przestrzenna\Projekt\Dane\dataset2.csv', index=False)

In [ ]:
#df_uaktualniony.to_csv(r'D:\Studia\4 rok\Przestrzenna\Projekt\Dane\dataset2.csv', index=False)